In [1]:
!pip install tensorflow

In [ ]:
# Step 1: Import Libraries and Enable Mixed Precision
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import matplotlib.pyplot as plt
import numpy as np
import time

# --- KEY FOR SPEED: Enable Mixed Precision Training ---
try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print("✅ Mixed precision enabled.")
except Exception as e:
    print(f"⚠️ Could not enable mixed precision: {e}")

# Record start time for performance measurement
start_time = time.time()

# Step 2: Load CIFAR-10 Dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Step 3: Create an Enhanced tf.data Pipeline
AUTOTUNE = tf.data.AUTOTUNE
BATCH_SIZE = 64
IMG_SIZE = 32

# --- NEW: More Powerful Data Augmentation ---
def random_cutout(image, mask_size_factor=0.5):
    """Applies Random Cutout augmentation."""
    input_shape = tf.shape(image)
    h, w = input_shape[0], input_shape[1]

    mask_h = tf.cast(tf.cast(h, tf.float32) * mask_size_factor, tf.int32)
    mask_w = tf.cast(tf.cast(w, tf.float32) * mask_size_factor, tf.int32)

    # Randomly select top-left corner
    offset_h = tf.random.uniform([], maxval=h - mask_h, dtype=tf.int32)
    offset_w = tf.random.uniform([], maxval=w - mask_w, dtype=tf.int32)

    # Create the cutout patch (zeros)
    cutout_patch = tf.zeros([mask_h, mask_w, 3], dtype=image.dtype)

    # Pad the cutout patch to image size
    padded_patch = tf.image.pad_to_bounding_box(cutout_patch, offset_h, offset_w, h, w)

    # Create a mask to apply cutout
    mask = tf.equal(padded_patch, 0)

    return tf.where(mask, image, 0)

def augment_data(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_brightness(image, max_delta=0.2)

    # Apply cutout with a 50% probability
    if tf.random.uniform([]) > 0.5:
        image = random_cutout(image)

    return image, label

# Create the datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=len(train_images))
train_dataset = train_dataset.map(augment_data, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Step 4: Build the Improved Model
def build_model():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # --- IMPROVED: Upsample to 64x64 for more detail ---
    # Ensure the size is an integer tuple
    x = layers.UpSampling2D(size=(64 // IMG_SIZE, 64 // IMG_SIZE), interpolation='bilinear')(inputs)

    base_model = applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_tensor=x,
        pooling='avg'
    )

    # --- IMPROVED: Progressive Fine-Tuning ---
    # Freeze the entire base model initially
    base_model.trainable = False
    # Unfreeze the top 20 layers for fine-tuning
    for layer in base_model.layers[-20:]:
        # The BatchNormalization layers should be kept frozen
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    # Build the classification head
    x = layers.BatchNormalization()(base_model.output)
    # --- IMPROVED: Increased dropout for better regularization ---
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10, activation='softmax', dtype='float32')(x)

    model = models.Model(inputs, outputs)
    return model

model = build_model()
model.summary()

# Step 5: Compile and Train with an Advanced Learning Rate Schedule
EPOCHS = 5 # Set a max number of epochs, EarlyStopping will handle the rest

# --- IMPROVED: Learning Rate Schedule for combined training ---
# Phase 1: Higher LR for 5 epochs to train the head
# Phase 2: Lower LR for the remaining epochs to fine-tune
boundaries = [5 * len(train_dataset)]
values = [1e-3, 1e-4]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

# Compile the model ONCE
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# --- IMPROVED: Single training run with EarlyStopping ---
# This stops training when validation loss stops improving, ensuring efficiency
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

history = model.fit(train_dataset,
                    epochs=EPOCHS,
                    validation_data=test_dataset,
                    callbacks=[early_stopping])

# Step 6: Evaluate the Final Model
print("\n--- Evaluating the final model on the test set ---")
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f"✅ Final Test Accuracy: {test_acc:.4f}")

end_time = time.time()
training_duration = end_time - start_time
print(f"⏱️ Total script duration: {training_duration / 60:.2f} minutes")

# Step 7: Plot Training History
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Step 8: Save Model Performance Report
try:
    best_epoch_idx = np.argmin(history.history['val_loss'])
    best_val_loss = history.history['val_loss'][best_epoch_idx]
    best_val_acc = history.history['val_accuracy'][best_epoch_idx]

    performance_text = f"""Model Performance Summary (Optimized EfficientNetB0):
Total Script Duration: {training_duration / 60:.2f} minutes

--- Final Evaluation on Test Set (Restored Best Weights) ---
Test Accuracy: {test_acc:.4f}
Test Loss: {test_loss:.4f}

--- Metrics at Best Epoch ({best_epoch_idx + 1}) ---
Best Validation Loss: {best_val_loss:.4f}
Best Validation Accuracy: {best_val_acc:.4f}

--- Training Details ---
Total Epochs Run: {len(history.history['loss'])}
Model: EfficientNetB0
Techniques: Transfer Learning, Progressive Fine-Tuning, Mixed Precision, Cutout Augmentation
Model Parameters: {model.count_params()}"""

    with open('model_accuracy_optimized.txt', 'w') as f:
        f.write(performance_text)

    print("\n📄 Model performance saved to model_accuracy_optimized.txt")
    print(performance_text)

except Exception as e:
    print(f"\n❌ Error saving model performance: {e}")